In [1]:
import requests
import subprocess
from lxml import html
from bs4 import BeautifulSoup

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
LOGIN_URL = "https://cs3500.ccs.neu.edu/users/sign_in"
REDIRECT_URL = "https://cs3500.ccs.neu.edu"

USERNAME = 'christopherwong'
PASSWORD = 'c8wong'

login_payload = {'user[username]': USERNAME,
                   'user[password]': PASSWORD}

In [3]:
# get session
s = requests.session()

# get login page
login = s.get(LOGIN_URL, verify=False)
login_html = html.fromstring(login.text)

# get csrf-token
hidden_inputs = login_html.xpath(r'//form//input[@type="hidden"]')
hidden_inputs[1].value
token = hidden_inputs[1].value
login_payload['authenticity_token'] = token

In [4]:
response = s.post(LOGIN_URL, 
                  data=login_payload,
                  headers = dict(referer=REDIRECT_URL))

In [5]:
# check if sign-in was successful
'Signed in successfully.' in response.text

True

In [6]:
# get link to submission page for first assignment in table
soup = BeautifulSoup(response.text, "lxml")
current_assignment = soup.find("td").a["href"]

In [7]:
# get link for new submission and get submission page
# SUBMISSION_PAGE_URL = REDIRECT_URL + current_assignment + "/submissions/new"
# submission_page = s.get(SUBMISSION_PAGE_URL, verify=False)

In [30]:
fileobj = open('submission.zip', 'rb')

# create new submission
submission_payload = {'submission[time_taken]': "5.0",
                      'submission[student_notes]': "Status code 0",
                      'submission[upload_file]': ("submission.zip", fileobj)}

In [51]:
# post submisison to page
SUBMISSION_POST_URL =  REDIRECT_URL + current_assignment + "/submissions/new" 
new_submission = s.post(SUBMISSION_POST_URL,
                        data=submission_payload,
                        headers = dict(referer=REDIRECT_URL))

print("Submission was successfully created." in new_submission.text)
print(new_submission.status_code)

False
404
